##A Google of Quotes

We want to build a search engine that, given a word or words, ﬁnds the best matching quotes.

Our approach was to first define a function (format_quotes()) which reads in the quotes2.txt file, strips down the text an creates a list of full quotes in the following form Quote – Author. To address part (b) we defined the words_from_quote() which takes in a quote and splits the quote into words, compiling a list of all lowercase words in the quote.
 
For part (c) we defined two functions. The first function (count_word_occurrence()) creates a dictionary from a quote with the keys as words in the quote and the values corresponding to the number of times the word appears in the given quote. The second function creates a posting list dictionary by calling the count word occurrence and format quotes functions. For part (d) we took a similar approach to part (c) but had to loop through each entry in the quote list to find the number of times a word appears in the dictionary. Next, we append this quote and count as a dictionary to the values and word to the keys of a new dictionary.
the

For part (e) we created three separate functions. The first calculates the term frequency. The second function calculates the inverse document frequency, and the third calls the first two and multiplies their outputs together, returning the TF-IDF measure for a word and a full quote.
 
For part (f) we wrote a function that takes in a word as an argument and calls the reverse-posting list dictionary. Next we indexed it and fed the index value to our TF-IDF function. This created our dictionary of full quotes containing the inputted word as keys and the corresponding TF-IDF measure as values. 


In [34]:
import math
import re
import collections

def format_quotes():

#  inputs: none
# outputs: list of quotes from quotes2.txt of form quote-speaker


    fp = open('quotes2.txt', 'r')
    line = fp.readline()
    quotes_list = []
    while line is not ' ':
        line = line.rstrip()
        new_line = fp.readline()
        new_line = new_line.rstrip()
        appended_line = '"' + line + '"' + " - " + new_line
        quotes_list.append(appended_line)
        line = fp.readline()
    fp.close()

    return quotes_list

In [35]:
def words_from_quote(quote):

#input: quote
#output: list of words

    list_of_words = re.split('\W+', quote.lower().replace('\'',"").replace('"',""))
    return list_of_words



In [36]:
def count_word_occurrence(quote):

#    input: quote
#    output: dictionary with keys word and values number of times word appears in quote

    d = {}
    for word in words_from_quote(quote):
        if word in d:
            d[word] += 1
        else:
            d[word] = 1
    return d
def create_postings_list_dictionary():

#    input: none
#    output: dictionary with quotes as keys and values are a dictionary with words in quote as keys and word count as values

    quotes_list = format_quotes()
    postings_list_dictionary = {}
    for quote in quotes_list:
        postings_list_dictionary[quote] = count_word_occurrence(quote)
    return postings_list_dictionary

In [37]:
def create_reverse_postings_list_dictionary():

#    input: none
#    output: dictionary with word as keys and values are dictionary of full quotes as keys and count of word in quote as values

    quotes_list = format_quotes()
    reverse_postings_list_dictionary = {}
    for quote in quotes_list:
        word_counts = count_word_occurrence(quote)
        for word, count in word_counts.items():
            if word in reverse_postings_list_dictionary.keys():
                dictionary_for_word = reverse_postings_list_dictionary[word]
                dictionary_for_word[quote] = count
            else:
                reverse_postings_list_dictionary[word] = {quote:count}
    return reverse_postings_list_dictionary

In [38]:
def term_frequency(word, quote):

#    input: word, quote
#    output: term frequency as float
  
    word_counts = count_word_occurrence(quote)
    term_frequency = float(word_counts[word]) / float(max(word_counts.values()))
    return term_frequency

def inverse_document_frequency(word, quote):
#    input: word, quote
#    output: inverse document frequency as a float  
    full_quotes_list = format_quotes()
    num_quotes = len(full_quotes_list)
    x = create_reverse_postings_list_dictionary()
    num_quotes_containing_word = len(x[word])
    IDF = math.log(float(num_quotes)/num_quotes_containing_word)
    return IDF
     
def TF_IDF(word, quote):
#    input: word, quote
#    output: TF multiplied by IDF to create TF_IDF measure
    x = term_frequency(word, quote)
    y = inverse_document_frequency(word, quote)
    return x * y

def TF_IDF_SUM(words, quote):
    sum_TFIDF = 0
    for word in words:
        sum_TFIDF += TF_IDF(word, quote)
    return sum_TFIDF

In [39]:
def single_word_search(word):

#    input: word
 #   output: dictionary with full quotes containing that word as key and TF_IDF scores as values

    x = create_reverse_postings_list_dictionary()
    output = x[word]
    for key in output:
        output[key] = TF_IDF(word, key)

    return output

In [40]:
def multiple_word_search(words):
    x = create_reverse_postings_list_dictionary()
    starting_point = x[words[0]]
    y = collections.OrderedDict(sorted(starting_point.items(), key = lambda t: t[0]))
    midpoint = [0]*len(y)
    for word in words:
        for index, quote in enumerate(y):
            if word in words_from_quote(quote):
                midpoint[index] += 1
    quotes_dict = {}
    for index, item in enumerate(midpoint):
        if item == len(words):
            quotes_dict[y.keys()[index]] = TF_IDF_SUM(words, y.keys()[index])
    return quotes_dict


Sample Run
 
Part (a)
 
No sample run generated because it is too long.
 
Part (b)
 
print words_from_quote('The world is full of suffering. It is also full of overcoming it. Helen Keller')
 
['the', 'world', 'is', 'full', 'of', 'suffering', 'it', 'is', 'also', 'full', 'of', 'overcoming', 'it', 'helen', 'keller']
 
Part (c)
create_postings_list_dictionary()
 
To shorten this response, we only should the sample run of the first entry in the postings-list dictionary
 
{'".I had to face the horrible truth: The antitobacco people are lying. Smoking really is cool. And I`m less cool for not doing it." - Tucker Carlson': {'': 1,
  'and': 1,
  'antitobacco': 1,
  'are': 1,
  'carlson': 1,
  'cool': 2,
  'doing': 1,
  'face': 1,
  'for': 1,
  'had': 1,
  'horrible': 1,
  'i': 2,
  'is': 1,
  'it': 1,
  'less': 1,
  'lying': 1,
  'm': 1,
  'not': 1,
  'people': 1,
  'really': 1,
  'smoking': 1,
  'the': 2,
  'to': 1,
  'truth': 1,
  'tucker': 1},
 
Part (d)
 
create_reverse_postings_list_dictionary()
 
To shorten this response, we only should the sample run for one entry in the reverse postings-list dictionary. The key for this entry is ‘longer’.
 
'longer': {'"I made this letter longer than usual because I lack the time to make it short." - Blaise Pascal': 1,
  '"Knowledge that is paid for will be longer remembered." - Nachman of Bratslav': 1,
  '"The illegal we do immediately. The unconstitutional takes a little longer." - Henry Kissinger': 1},
 
Part (e)
 
TF_IDF('a', 'A right delayed is a right denied. - Martin Luther King Jr.')
1.0090646273824244
 
Part (f)
single_word_search('government')
 
{'"A government which robs Peter to pay Paul can always depend on the support of Paul." - George Bernard Shaw': 1.871097256440829,
 '"An oppressive government is more to be feared than a tiger." - Confucius': 3.742194512881658,
 '"Are you entitled to the fruits of your labor or does government have some presumptive right to spend and spend and spend?" - Ronald Reagan': 1.2473981709605526,
 '"As Mankind becomes more liberal, they will be more apt to allow that all those who conduct themselves as worthy members of the community are equally entitled to the protections of civil government. I hope ever to see America among the foremost nations of justice and liberality." - George Washington': 1.2473981709605526,
 '"Be thankful we`re not getting all the government we`re paying for." - Will Rogers': 1.871097256440829,
 '"Every decent man is ashamed of the government he lives under." - Henry Mencken': 3.742194512881658,
 '"Experts tell us that if the Millennium Bug is not fixed, when the year 2000 arrives, our financial records will be inaccurate, our telephone system will be unreliable, our government will be paralyzed and airline flights will be canceled without warning. In other words, things will be pretty much the same as they are now." - Dave Barry': 0.7484389025763316,
 '"Government consists in nothing else but so controlling subjects that they shall neither be able to, nor have cause to do it harm." - Nicolo Machiavelli': 1.871097256440829,
 '"Government is not a solution to our problem, government is the problem." - Ronald Reagan': 3.742194512881658,
 '"Government is not reason; it is not eloquent; it is force. Like fire, it is a dangerous servant and a fearful master." - George Washington': 0.9355486282204145,
 '"I believe this government cannot endure permanently half slave and half free." - Abraham Lincoln': 1.871097256440829,
 '"I don`t make jokes. I just watch the government and report the facts." - Will Rogers': 1.871097256440829,
 '"I`m a middle age white guy, which means I`m constantly reminded that my particular group is responsible for the oppression of every known minority PLUS most wars PLUS government corruption PLUS pollution of the environment, not to mention that it was middle-age white guys who killed Bambi`s mom." - Dave Barry': 1.2473981709605526,
 '"In framing a government which is to be administered by men over men the great difficulty lies in this: You must first enable the government to control the governed, and in the next place, oblige it to control itself." - Alexander Hamilton': 1.871097256440829,
 '"In the councils of government, we must guard against the acquisition of unwarranted influence, whether sought or unsought, by the military-industrial complex." - Dwight Eisenhower': 1.2473981709605526,
 '"It is error alone which needs the support of government. Truth can stand by itself." - Thomas Jefferson': 3.742194512881658,
 '"On account of being a democracy and run by the people, we are the only nation in the world that has to keep a government four years, no matter what it does." - Will Rogers': 1.2473981709605526,
 '"The best defense against usurpatory government is an assertive citizenry." - William F. Buckley': 3.742194512881658,
 '"Trust the people. This is the one irrefutable lesson of the entire postwar period contradicting the notion that rigid government controls are essential to economic development." - Ronald Reagan': 0.9355486282204145,
 '"Under the doctrine of the separation of powers, the manner in which the president personally exercises his assigned executive powers is not subject to questioning by another branch of government." - Richard Nixon': 0.9355486282204145,
 '"Without computers, the government would be unable to function at the level of effectiveness and efficiency that we have come to expect.  This is because the primary function of the government is -- and here I am quoting directly from the U.S. Constitution -- `to spew out paper.`" - Dave Barry': 1.4968778051526632}
 
Part (g)
 
multiple_word_search(['government', 'solution'])
 
{'"Government is not a solution to our problem, government is the problem." - Ronald Reagan': 7.135552988184198}